# Predicting Wine Quality

Following tutorial: https://elitedatascience.com/python-machine-learning-tutorial-scikit-learn

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
#Tools for cross-validation
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
#Tools for model evaluation
from sklearn.metrics import mean_squared_error, r2_score
#model persistence for future use
from sklearn.externals import joblib

In [5]:
#Tip: read_csv() can work for even remote URL
dataset_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'

df = pd.read_csv(dataset_url)

In [6]:
df.head()#data is separated using semi-colons

,"fixed acidity;""volatile acidity"";""citric acid"";""residual sugar"";""chlorides"";""free sulfur dioxide"";""total sulfur dioxide"";""density"";""pH"";""sulphates"";""alcohol"";""quality"""
0,7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5
1,7.8;0.88;0;2.6;0.098;25;67;0.9968;3.2;0.68;9.8;5
2,7.8;0.76;0.04;2.3;0.092;15;54;0.997;3.26;0.65;...
3,11.2;0.28;0.56;1.9;0.075;17;60;0.998;3.16;0.58...
4,7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5


In [9]:
df = pd.read_csv(dataset_url, sep = ';')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [12]:
df.shape

(1599, 12)

In [13]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [16]:
list(df.columns)#all features are numeric, but at very different scales
#in order to eliminate affect of differences in scale, we must standardize the data


['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [19]:
y = df.quality
X = df.drop('quality', axis = 1)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123, stratify = y)
#We stratify our sample by the target variable.
#This ensures training set looks seimilar to test set, increasing reliability.

### What is standarization?
"Standardization is the process of subtracting the means from each feature and then dividing by the feature standard deviations.

Standardization is a common requirement for machine learning tasks. Many algorithms assume that all features are centered around zero and have approximately the same variance."

Steps: 
1. Fit the transformer on the training set (saving the means and standard deviations)
2. Apply the transformer to the training set (scaling the training data)
3. Apply the transformer to the test set (using the same means and standard deviations)

In [28]:
scaler = preprocessing.StandardScaler().fit(X_train)
#scaler object has the saved means and std for each feature in training set
#scaler makes mean 0, std 1

In [29]:
X_train_scaled = scaler.transform(X_train)
print(X_train_scaled.mean(axis = 0))
print(X_train_scaled.std(axis = 0))

[ 1.16664562e-16 -3.05550043e-17 -8.47206937e-17 -2.22218213e-17
  2.22218213e-17 -6.38877362e-17 -4.16659149e-18 -2.54439854e-15
 -8.70817622e-16 -4.08325966e-16 -1.17220107e-15]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [31]:
#We're going ot us the same scaler object to transform the test set,
    #using same exact means and std used to transform training set
X_test_scaled = scaler.transform(X_test)
print(X_test_scaled.mean(axis = 0))
print(X_test_scaled.std(axis = 0))
"""
    We expect the mean to not be perfectly zero nor unit variance
This is because we're transforming test set using means from the
training set, not the test set itself
"""

[ 0.02776704  0.02592492 -0.03078587 -0.03137977 -0.00471876 -0.04413827
 -0.02414174 -0.00293273 -0.00467444 -0.10894663  0.01043391]
[1.02160495 1.00135689 0.97456598 0.91099054 0.86716698 0.94193125
 1.03673213 1.03145119 0.95734849 0.83829505 1.0286218 ]


"\n    We expect the mean to not be perfectly zero nor unit variance\nThis is because we're transforming test set using means from the\ntraining set, not the test set itself\n"

In [34]:
#In practice, we won't need to do the above practice manually
#This is a modeling pipeline that first transforms the data using
    #StandardScaler() and then fits a model using random forest regressor
pipeline = make_pipeline(preprocessing.StandardScaler(),
                        RandomForestRegressor(n_estimators = 100))

## Hyperparameters

What are they: 'higher-level' structural information about the model. They cannot be learned directly from the data, unlike model parameters

In [44]:
#listing tunable hyperparameters
print(pipeline.get_params())

{'memory': None, 'steps': [('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False))], 'verbose': False, 'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'randomforestregressor': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
           

In [48]:
#declaring hyperparams we want to tune
#this is a python dictionary. Keys are hyperparm names, and values are lists of settings to try
hyperparams = {'randomforestregressor__max_features':['auto','sqrt','log2'],
              'randomforestregressor__max_depth':[None, 5, 3, 1]}

"""
Epiphany: This is a much easier way than using tedious loops for 
testing many options of a feature. I suppose this tuning is
automated?
"""

'\nEpiphany: This is a much easier way than using tedious loops for \ntesting many options of a feature. I suppose this tuning is\nautomated?\n'

## Tune model using a cross-validation pipeline

Cross-validation is important. It helps us maximize performance while reduce chance of overfitting.

What is Cross-Validation??
    "The process for reliably estimating the performance of a method for building a model by training and evaluating your model multiple times using the same method" In our context, a "method" is simply a set of hyperparams.
    
Steps for Cross-Validation (CV):
1. Split your data into k equal parts, or "folds" (typically k=10).
2. Train your model on k-1 folds (e.g. the first 9 folds).
3. Evaluate it on the remaining "hold-out" fold (e.g. the 10th fold).
4. Perform steps (2) and (3) k times, each time holding out a different fold.
5. Aggregate the performance across all k folds. This is your performance metric.

Best practice when performing CV is to include data preprocessing steps inside cross-validation loop. Prevents accidentally tainting training folds with influential data from test fold. This is referred to as cross-validation "pipeline".

CV pipeline with preprocessing steps:
1. Split your data into k equal parts, or "folds" (typically k=10).
2. Preprocess k-1 training folds.
3. Train your model on the same k-1 folds.
4. Preprocess the hold-out fold using the same transformations from step (2).
5. Evaluate your model on the same hold-out fold.
6. Perform steps (2) - (5) k times, each time holding out a different fold.
7. Aggregate the performance across all k folds. This is your performance metric.
    

In [52]:
#With Scikit-Learn this is super easy. cv is number of folds.
clf = GridSearchCV(pipeline, hyperparams, cv = 10)

#Fit and tune model
clf.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('randomforestregressor',
                                        RandomForestRegressor(bootstrap=True,
                                                              criterion='mse',
                                                              max_depth=None,
                                                              max_features='auto',
                                                              max_leaf_nodes=None,
                                                              min_impurity_decrease=0.0,
                                                              min_impurity_split

In [53]:
print(clf.best_params_)

{'randomforestregressor__max_depth': None, 'randomforestregressor__max_features': 'log2'}


### Once we determine best parameters, we refit on entire training data set.

The best parameters were determined by using the cross-validation method described above. This means we used a fold of our training data to test for the best parameters. Now we know the best parameters, we refit on the entire training data set.

In [55]:
print(clf.refit)
#we can use our clf directly like a model object. yay!

True


In [56]:
y_pred = clf.predict(X_test)

In [60]:
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_pred, y_test))

0.47261795508202686
0.34030562499999994


### Tips for Improving Model Performance

1. Try other regression model families (e.g. regularized regression, boosted trees, etc.).
2. Collect more data if it's cheap to do so.
3. Engineer smarter features after spending more time on exploratory analysis.
4. Speak to a domain expert to get more context (...this is a good excuse to go wine tasting!)

### Saving model for future use

In [61]:
joblib.dump(clf, 'rf_regressor.pkl')

['rf_regressor.pkl']

In [62]:
#To load model, simply use:
clf2 = joblib.load('rf_regressor.pkl')

In [63]:
clf2.predict(X_test)

array([6.46, 5.79, 5.  , 5.43, 6.38, 5.42, 5.04, 4.82, 5.02, 6.13, 5.32,
       5.67, 5.84, 5.04, 5.88, 5.57, 6.56, 5.78, 5.75, 6.96, 5.48, 5.7 ,
       4.97, 6.02, 5.93, 5.06, 5.51, 5.19, 5.84, 5.89, 5.87, 6.4 , 5.99,
       5.02, 5.  , 5.88, 5.02, 6.07, 5.21, 5.98, 5.  , 5.99, 6.77, 5.16,
       6.14, 5.35, 5.51, 5.56, 5.17, 6.44, 6.04, 5.32, 5.78, 5.18, 5.55,
       5.92, 5.33, 5.34, 4.99, 5.35, 5.38, 5.2 , 5.08, 5.81, 5.99, 5.27,
       6.24, 5.04, 5.27, 6.7 , 5.6 , 5.92, 5.06, 4.99, 5.31, 5.99, 5.39,
       5.1 , 5.34, 5.28, 6.34, 5.49, 6.13, 6.32, 5.08, 5.95, 6.44, 6.34,
       5.74, 5.59, 5.93, 5.33, 6.36, 5.68, 5.72, 5.79, 6.67, 6.73, 5.56,
       6.79, 5.1 , 5.39, 5.14, 6.41, 5.07, 4.73, 5.74, 4.99, 5.71, 5.95,
       5.95, 5.39, 6.  , 5.44, 5.06, 5.24, 5.91, 5.2 , 4.95, 5.98, 5.81,
       5.12, 5.78, 6.12, 5.28, 5.31, 5.35, 6.01, 5.35, 5.45, 5.59, 6.19,
       5.17, 5.32, 5.07, 6.42, 5.05, 5.1 , 6.66, 5.46, 5.13, 5.06, 5.65,
       6.09, 5.39, 5.36, 5.11, 6.4 , 5.84, 5.18, 5.